In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.utils import class_weight
from imblearn.over_sampling import RandomOverSampler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.preprocessing import PolynomialFeatures

In [7]:
# Read & Adjust Data from CSV file

# We are trying here linear and 2nd, and 3rd order polynomial regression with oversampling our data to compromise for the inbalance.
# We also applied pca  (a trail to improve performance) on scaled data (as it affects pca severily), to decrease its huge dimensions. As, it's very suspecious to have usless/correlated features.

data = pd.read_csv("C:/Users/Mahmoud/Documents/GitHub/Machine-Learning-Classification-Project/Preprocessing/2017_Accidents_UK_Clean _Mahmoud.csv",dtype=float)
x = data.iloc[:, :-1] 
y = data.iloc[:, -1]  

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

priciple_component_analayzer = PCA(n_components=15)  
x_pca = priciple_component_analayzer.fit_transform(x_scaled)

oversampler = RandomOverSampler(random_state=10)
x_resampled, y_resampled = oversampler.fit_resample(x_pca, y)
x=pd.DataFrame(x_resampled)
y=y_resampled

x_training, x_testing, y_training, y_testing = train_test_split(x, y, test_size=0.15, random_state=10)
x=x_training
y=y_training

In [8]:
# We try different parameters and apply K-folding on them. Our evaluation metrices will be accuracy, precision, and recall.
# We weren't able to apply ROC, as it's a multi-class output, and having 1v2 roc isn't the best metric. 
# We store the results to choose the best one later.
# This is the linear one.

penalty_options = ['l1',None, 'l2']
C_values = [0.001, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64, 1.28, 2.56, 5.12, 10.24]
solver_options = {
    'lbfgs': ['l2', None],
    'liblinear': ['l1', 'l2'],
    'newton-cg': ['l2', None],
    'newton-cholesky': ['l2', None],
    'sag': ['l2', None],
    'saga': ['elasticnet', 'l1', 'l2', None]
}
results = []
warnings.filterwarnings("ignore", category=ConvergenceWarning)

for penalty in penalty_options:
    for C in C_values:
        for solver in solver_options.keys():
            if penalty in solver_options[solver]:
                accuracy_scores = []
                precision_scores = []
                recall_scores = []

                kf = StratifiedKFold(n_splits=5, random_state=10, shuffle=True)

                for train_index, test_index in kf.split(x, y):
                    X_train_fold, X_cross = x.iloc[train_index], x.iloc[test_index]
                    y_train_fold, y_cross = y.iloc[train_index], y.iloc[test_index]
                    
                    logistic_model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=100)
                    logistic_model.fit(X_train_fold, y_train_fold)
                    y_pred = logistic_model.predict(X_cross)

                    accuracy = accuracy_score(y_cross, y_pred)
                    precision = precision_score(y_cross, y_pred, average='macro', zero_division=0)
                    recall = recall_score(y_cross, y_pred, average='macro')

                    accuracy_scores.append(accuracy)
                    precision_scores.append(precision)
                    recall_scores.append(recall)

                this_result = {
                    'penalty': penalty,
                    'C': C,
                    'solver': solver,
                    'accuracy': np.mean(accuracy_scores),
                    'precision': np.mean(precision_scores),
                    'recall': np.mean(recall_scores)
                }
                results.append(this_result)
                print(this_result)


{'penalty': 'l1', 'C': 0.001, 'solver': 'liblinear', 'accuracy': 0.47939477914598305, 'precision': 0.46828655015585685, 'recall': 0.47942007993648617}
{'penalty': 'l1', 'C': 0.001, 'solver': 'saga', 'accuracy': 0.47976562373480885, 'precision': 0.47031824838950503, 'recall': 0.4797871769347588}
{'penalty': 'l1', 'C': 0.01, 'solver': 'liblinear', 'accuracy': 0.4797070654402941, 'precision': 0.4693070081194829, 'recall': 0.4797309528173274}
{'penalty': 'l1', 'C': 0.01, 'solver': 'saga', 'accuracy': 0.48071810562968664, 'precision': 0.4719775535784977, 'recall': 0.48073874451745924}
{'penalty': 'l1', 'C': 0.02, 'solver': 'liblinear', 'accuracy': 0.4798280776869716, 'precision': 0.46944187784226754, 'recall': 0.4798519010742358}
{'penalty': 'l1', 'C': 0.02, 'solver': 'saga', 'accuracy': 0.4805190200760808, 'precision': 0.47181589379621214, 'recall': 0.48053957774129447}
{'penalty': 'l1', 'C': 0.04, 'solver': 'liblinear', 'accuracy': 0.47985540336725396, 'precision': 0.4694857952203888, 're

C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


{'penalty': None, 'C': 0.001, 'solver': 'lbfgs', 'accuracy': 0.48034335669741246, 'precision': 0.47168689300303424, 'recall': 0.480363850058697}


C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


{'penalty': None, 'C': 0.001, 'solver': 'newton-cg', 'accuracy': 0.4802848032029453, 'precision': 0.4716356968378765, 'recall': 0.4803052853634047}


...

{'penalty': 'l2', 'C': 1.28, 'solver': 'newton-cg', 'accuracy': 0.4802848032029453, 'precision': 0.4716356968378765, 'recall': 0.4803052853634047}
{'penalty': 'l2', 'C': 1.28, 'solver': 'newton-cholesky', 'accuracy': 0.4799959339787591, 'precision': 0.4696405408282721, 'recall': 0.48001968385635596}
{'penalty': 'l2', 'C': 1.28, 'solver': 'sag', 'accuracy': 0.4802887066320989, 'precision': 0.47163874925739224, 'recall': 0.4803091903924136}
{'penalty': 'l2', 'C': 1.28, 'solver': 'saga', 'accuracy': 0.4802848030505628, 'precision': 0.4716339784358393, 'recall': 0.4803052876487849}
{'penalty': 'l2', 'C': 2.56, 'solver': 'lbfgs', 'accuracy': 0.48034726027894853, 'precision': 0.471690839465032, 'recall': 0.4803677539450158}
{'penalty': 'l2', 'C': 2.56, 'solver': 'liblinear', 'accuracy': 0.4799842231579598, 'precision': 0.46962544066793493, 'recall': 0.48000797562546965}


C:\Users\Mahmoud\anaconda3\lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


{'penalty': 'l2', 'C': 2.56, 'solver': 'newton-cg', 'accuracy': 0.4802848032029453, 'precision': 0.4716356968378765, 'recall': 0.4803052853634047}
{'penalty': 'l2', 'C': 2.56, 'solver': 'newton-cholesky', 'accuracy': 0.4799959339787591, 'precision': 0.4696405408282721, 'recall': 0.48001968385635596}
{'penalty': 'l2', 'C': 2.56, 'solver': 'sag', 'accuracy': 0.48034335799266337, 'precision': 0.4716823528379727, 'recall': 0.48036385485973526}
{'penalty': 'l2', 'C': 2.56, 'solver': 'saga', 'accuracy': 0.48024186296794513, 'precision': 0.4715955376920345, 'recall': 0.48026233758400017}
{'penalty': 'l2', 'C': 5.12, 'solver': 'lbfgs', 'accuracy': 0.48034335669741246, 'precision': 0.47168689300303424, 'recall': 0.480363850058697}
{'penalty': 'l2', 'C': 5.12, 'solver': 'liblinear', 'accuracy': 0.4799842231579598, 'precision': 0.46962544066793493, 'recall': 0.48000797562546965}


C:\Users\Mahmoud\anaconda3\lib\site-packages\scipy\optimize\_linesearch.py:306: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Mahmoud\anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


{'penalty': 'l2', 'C': 5.12, 'solver': 'newton-cg', 'accuracy': 0.4802848032029453, 'precision': 0.4716356968378765, 'recall': 0.4803052853634047}
{'penalty': 'l2', 'C': 5.12, 'solver': 'newton-cholesky', 'accuracy': 0.4799959339787591, 'precision': 0.4696405408282721, 'recall': 0.48001968385635596}
{'penalty': 'l2', 'C': 5.12, 'solver': 'sag', 'accuracy': 0.4805619602348898, 'precision': 0.47188622091282584, 'recall': 0.4805824862120004}
{'penalty': 'l2', 'C': 5.12, 'solver': 'saga', 'accuracy': 0.4802926103660174, 'precision': 0.47164271491569004, 'recall': 0.4803130942787325}
{'penalty': 'l2', 'C': 10.24, 'solver': 'lbfgs', 'accuracy': 0.48034335669741246, 'precision': 0.47168689300303424, 'recall': 0.480363850058697}
{'penalty': 'l2', 'C': 10.24, 'solver': 'liblinear', 'accuracy': 0.4799842231579598, 'precision': 0.46962544066793493, 'recall': 0.48000797562546965}
{'penalty': 'l2', 'C': 10.24, 'solver': 'newton-cg', 'accuracy': 0.4802848032029453, 'precision': 0.4716356968378765, '

In [14]:
# We favor precision and recall over accuracy ; however, we still take accuracy into consideration. 
# So we take the top 10 accuracy scores passing the average precision and recall values.
# After this we, take the top 5 of them based on precision. Then we take the one having the maximum recall.

precision_values = [result['precision'] for result in results]
all_models_average_precision_scores = np.mean(precision_values)

recall_values = [result['recall'] for result in results]
all_models_average_recall_scores = np.mean(recall_values)

filtered_results = []
for result in results:
    if result['precision'] >= all_models_average_precision_scores and result['recall'] >= all_models_average_recall_scores :
        filtered_results.append(result)
        
top_10_results = sorted(filtered_results, key=lambda x: x['accuracy'], reverse=True)[:10]

precision_values = [result['precision'] for result in top_10_results]
top_10_results_average_precision_scores = np.mean(precision_values)


filtered_results_2 = []
for result in top_10_results:
    if result['precision'] >= top_10_results_average_precision_scores:
        filtered_results_2.append(result)
top_5_results = sorted(filtered_results_2, key=lambda x: x['precision'], reverse=True)[:5]

best_model_1 = max(top_5_results, key=lambda x: x['recall'])

print(best_model_1)

{'penalty': 'l2', 'C': 0.001, 'solver': 'newton-cg', 'accuracy': 0.4808898651982452, 'precision': 0.4721973939232377, 'recall': 0.48091046181009905}


In [16]:
# We try different parameters and apply K-folding on them. Our evaluation metrices will be accuracy, precision, and recall.
# We weren't able to apply ROC, as it's a multi-class output, and having 1v2 roc isn't the best metric. 
# We store the results to choose the best one later.
# This is the 2nd order one; however, we interrupted the kernel, as it was showing very slight improvement, and we already tried
# this on weighting data in the other file, and got no improvement. It took there lots of time, so we didn't wanna go through the same thing.

penalty_options = ['l1', None, 'l2']
C_values = [0.001, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64, 1.28, 2.56, 5.12, 10.24]
solver_options = {
    'lbfgs': ['l2', None],
    'liblinear': ['l1', 'l2'],
    'newton-cg': ['l2', None],
    'newton-cholesky': ['l2', None],
    'sag': ['l2', None],
    'saga': ['elasticnet', 'l1', 'l2', None]
}
results = []
warnings.filterwarnings("ignore", category=ConvergenceWarning)

polynomial_transformer = PolynomialFeatures(degree=2)
x_poly = polynomial_transformer.fit_transform(x)  

for penalty in penalty_options:
    for C in C_values:
        for solver in solver_options.keys():
            if penalty in solver_options[solver]:
                accuracy_scores = []
                precision_scores = []
                recall_scores = []

                kf = StratifiedKFold(n_splits=5, random_state=10, shuffle=True)

                for train_index, test_index in kf.split(x_poly, y):  # Use the polynomial features (x_poly)
                    X_train_fold, X_cross = x_poly[train_index], x_poly[test_index]
                    y_train_fold, y_cross = y.iloc[train_index], y.iloc[test_index]

                    logistic_model = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=100, class_weight=class_weight_dictionary)
                    logistic_model.fit(X_train_fold, y_train_fold)
                    y_pred = logistic_model.predict(X_cross)

                    accuracy = accuracy_score(y_cross, y_pred)
                    precision = precision_score(y_cross, y_pred, average='macro', zero_division=0)
                    recall = recall_score(y_cross, y_pred, average='macro')

                    accuracy_scores.append(accuracy)
                    precision_scores.append(precision)
                    recall_scores.append(recall)

                this_result = {
                    'penalty': penalty,
                    'C': C,
                    'solver': solver,
                    'accuracy': np.mean(accuracy_scores),
                    'precision': np.mean(precision_scores),
                    'recall': np.mean(recall_scores)
                }
                results.append(this_result)
                print(this_result)


{'penalty': 'l1', 'C': 0.001, 'solver': 'liblinear', 'accuracy': 0.4965628554666174, 'precision': 0.4850331298520678, 'recall': 0.4965884756630777}
{'penalty': 'l1', 'C': 0.001, 'solver': 'saga', 'accuracy': 0.4960983256828385, 'precision': 0.4863139564034234, 'recall': 0.49612051884594405}
{'penalty': 'l1', 'C': 0.01, 'solver': 'liblinear', 'accuracy': 0.5036479120313112, 'precision': 0.4925271514221919, 'recall': 0.5036724848137919}
{'penalty': 'l1', 'C': 0.01, 'solver': 'saga', 'accuracy': 0.5024494997758053, 'precision': 0.49198789172499335, 'recall': 0.5024728340782166}
{'penalty': 'l1', 'C': 0.02, 'solver': 'liblinear', 'accuracy': 0.5042568797415018, 'precision': 0.49317626903345263, 'recall': 0.50428141909731}
{'penalty': 'l1', 'C': 0.02, 'solver': 'saga', 'accuracy': 0.5030155298414976, 'precision': 0.49255395073133157, 'recall': 0.5030388964747246}
{'penalty': 'l1', 'C': 0.04, 'solver': 'liblinear', 'accuracy': 0.5052054532547958, 'precision': 0.49408138015092185, 'recall': 0

KeyboardInterrupt: 

In [17]:
# We favor precision and recall over accuracy ; however, we still take accuracy into consideration. 
# So we take the top 10 accuracy scores passing the average precision and recall values.
# After this we, take the top 5 of them based on precision. Then we take the one having the maximum recall.

precision_values = [result['precision'] for result in results]

all_models_average_precision_scores = np.mean(precision_values)

recall_values = [result['recall'] for result in results]
all_models_average_recall_scores = np.mean(recall_values)

filtered_results = []
for result in results:
    if result['precision'] >= all_models_average_precision_scores and result['recall'] >= all_models_average_recall_scores:
        filtered_results.append(result)     
top_10_results = sorted(filtered_results, key=lambda x: x['accuracy'], reverse=True)[:10]

precision_values = [result['precision'] for result in top_10_results]
top_10_results_average_precision_scores = np.mean(precision_values)


filtered_results_2 = []
for result in top_10_results:
    if result['precision'] >= top_10_results_average_precision_scores:
        filtered_results_2.append(result)
top_5_results = sorted(filtered_results_2, key=lambda x: x['precision'], reverse=True)[:5]

best_model_2 = max(top_5_results, key=lambda x: x['recall'])

print(best_model_2)

{'penalty': 'l1', 'C': 0.32, 'solver': 'liblinear', 'accuracy': 0.5063335989854894, 'precision': 0.4951384724878055, 'recall': 0.5063583383101693}


In [18]:
#Compare betwen the 2 models (2nd, and first order polynomial logistic regression models )

print(best_model_1)
print(best_model_2)


{'penalty': 'l2', 'C': 0.001, 'solver': 'newton-cg', 'accuracy': 0.4808898651982452, 'precision': 0.4721973939232377, 'recall': 0.48091046181009905}
{'penalty': 'l1', 'C': 0.32, 'solver': 'liblinear', 'accuracy': 0.5063335989854894, 'precision': 0.4951384724878055, 'recall': 0.5063583383101693}


In [19]:
# Create the final model with the best parameters

# On trying lots of different parameters, the model always performs poorly.
# Here is some suggestions that may improve the performance.
# Logistic regression may be affected to outliers, as it's very sensitive to them so using a clustering algorithm to kick outliers, and grouping similar data.
# Also trying different thresholds, adding/removing some features (domain knowledge), or trying even higher order models.
# However, at this point I think moving to another algorithm is more efficient, as they already show way better results, and
# this models is computationally expensive, and takes too long to train.

final_logistic_model = LogisticRegression(penalty=best_model_1['penalty'], C=best_model_1['C'], solver=best_model_1['solver']
                    , max_iter=5000, class_weight=class_weight_dictionary)

final_logistic_model.fit(x,y)
y_prediction = final_logistic_model.predict(x_testing)

final_accuracy = accuracy_score(y_testing, y_prediction)
final_precision = precision_score(y_testing, y_prediction, average='macro',zero_division=0)
final_recall = recall_score(y_testing, y_prediction, average='macro')
my_confusion_matrix = confusion_matrix(y_testing, y_prediction, labels=[0, 1, 2])

print("Testing Accuracy:", final_accuracy)
print("Testing Precision:", final_precision)
print("Testing Recall:", final_recall)

print("Confusion Matrix:")
print("True Fatal: " , my_confusion_matrix[0,0],  "False Serious: ", my_confusion_matrix[0,1], "False Slight: ",my_confusion_matrix[0,2] )
print("False Fatal: " , my_confusion_matrix[1,0],  "True Serious: ", my_confusion_matrix[1,1], "False Slight: ",my_confusion_matrix[1,2])
print("False Fatal: " , my_confusion_matrix[2,0],  "False Serious: ", my_confusion_matrix[2,1], "True Slight: ",my_confusion_matrix[2,2])


# confusion matrix explanation (applies to more dimensions)
#                  Predicted bird                         Predicted cat
# Actual bird       True bird                               False Cat 
# Actual cat        False Bird                             True Cat  

Testing Accuracy: 0.4825801313955803
Testing Precision: 0.47362605679348485
Testing Recall: 0.48246178167525694
Confusion Matrix:
True Fatal:  9416 False Serious:  3010 False Slight:  2656
False Fatal:  5095 True Serious:  4638 False Slight:  5317
False Fatal:  3518 False Serious:  3795 True Slight:  7762
